In [51]:
import pandas as pd

gc = pd.read_csv("game_data.csv")

ball_data = gc[gc["class"] == 1]

ball_data['ball'] = ball_data.groupby(['time', 'team']).cumcount() + 1
ball_data = ball_data[["time", "x_pos", "y_pos"]]
ball_data = ball_data.reset_index(drop=True)

gc = gc[gc["class"] == 0]
gc['robot'] = gc.groupby(['time', 'team']).cumcount() + 1
gc = gc.groupby(["time", "team"]).apply(lambda x: x.reset_index(drop=True))

pivot_table = gc.pivot(index=['time', 'team'], columns=
                       "robot", values=['x_pos', 'y_pos'])

pivot_table.columns = [f'{col[0]}_{col[1]}' for col in pivot_table.columns]

pivot_table.reset_index(inplace=True)

merged = pd.merge(pivot_table, ball_data, on="time", how="inner")
merged = merged.rename(columns={"x_pos": "x_pos_ball", "y_pos": "y_pos_ball"})
merged

/tmp/ipykernel_26322/2445375905.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ball_data['ball'] = ball_data.groupby(['time', 'team']).cumcount() + 1


,time,team,x_pos_1,x_pos_2,x_pos_3,x_pos_4,x_pos_5,x_pos_6,x_pos_7,y_pos_1,y_pos_2,y_pos_3,y_pos_4,y_pos_5,y_pos_6,y_pos_7,x_pos_ball,y_pos_ball
0,5,0,198.0,222.0,314.0,256.0,77.0,74.0,NaN,340.0,94.0,206.0,267.0,215.0,251.0,NaN,348,196
1,5,1,432.0,596.0,459.0,565.0,482.0,633.0,429.0,127.0,265.0,296.0,226.0,200.0,241.0,410.0,348,196
2,6,0,222.0,198.0,77.0,256.0,74.0,314.0,NaN,94.0,340.0,215.0,267.0,251.0,206.0,NaN,348,196
3,6,1,429.0,481.0,633.0,459.0,596.0,432.0,565.0,410.0,200.0,241.0,297.0,265.0,128.0,226.0,348,196
4,7,0,314.0,74.0,222.0,77.0,198.0,256.0,NaN,206.0,251.0,94.0,215.0,340.0,267.0,NaN,348,196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1665,1008,1,411.0,587.0,573.0,601.0,536.0,412.0,NaN,429.0,192.0,417.0,353.0,407.0,339.0,NaN,599,312
1666,1009,0,209.0,78.0,394.0,257.0,635.0,NaN,NaN,253.0,238.0,210.0,346.0,256.0,NaN,NaN,600,311
1667,1009,1,573.0,411.0,601.0,587.0,412.0,NaN,NaN,417.0,429.0,353.0,192.0,339.0,NaN,NaN,600,311
1668,1010,0,395.0,257.0,209.0,78.0,NaN,NaN,NaN,210.0,346.0,253.0,236.0,NaN,NaN,NaN,602,309


In [52]:
ballControlThreshold = 40

def ballControl(row):
    l = list()
    for robot in range(1,8):
        l.append( ((row[f'x_pos_{robot}'] - row['x_pos_ball'])**2 +(row[f'y_pos_{robot}'] - row['y_pos_ball'])**2)**0.5)
    
    return int(min(l) < ballControlThreshold)


merged['ballControl'] = merged.apply(ballControl, axis=1)

(merged["ballControl"]==True).sum()/merged.shape[0]

0.4940119760479042

In [53]:
merged.to_csv("merged.csv")
merged

,time,team,x_pos_1,x_pos_2,x_pos_3,x_pos_4,x_pos_5,x_pos_6,x_pos_7,y_pos_1,y_pos_2,y_pos_3,y_pos_4,y_pos_5,y_pos_6,y_pos_7,x_pos_ball,y_pos_ball,ballControl
0,5,0,198.0,222.0,314.0,256.0,77.0,74.0,NaN,340.0,94.0,206.0,267.0,215.0,251.0,NaN,348,196,1
1,5,1,432.0,596.0,459.0,565.0,482.0,633.0,429.0,127.0,265.0,296.0,226.0,200.0,241.0,410.0,348,196,0
2,6,0,222.0,198.0,77.0,256.0,74.0,314.0,NaN,94.0,340.0,215.0,267.0,251.0,206.0,NaN,348,196,1
3,6,1,429.0,481.0,633.0,459.0,596.0,432.0,565.0,410.0,200.0,241.0,297.0,265.0,128.0,226.0,348,196,0
4,7,0,314.0,74.0,222.0,77.0,198.0,256.0,NaN,206.0,251.0,94.0,215.0,340.0,267.0,NaN,348,196,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1665,1008,1,411.0,587.0,573.0,601.0,536.0,412.0,NaN,429.0,192.0,417.0,353.0,407.0,339.0,NaN,599,312,0
1666,1009,0,209.0,78.0,394.0,257.0,635.0,NaN,NaN,253.0,238.0,210.0,346.0,256.0,NaN,NaN,600,311,0
1667,1009,1,573.0,411.0,601.0,587.0,412.0,NaN,NaN,417.0,429.0,353.0,192.0,339.0,NaN,NaN,600,311,0
1668,1010,0,395.0,257.0,209.0,78.0,NaN,NaN,NaN,210.0,346.0,253.0,236.0,NaN,NaN,NaN,602,309,0


In [47]:

field_start_x = 0
field_end_x = 690
field_start_y = 0
field_end_y = 464

for column in merged.columns:
    if column.startswith('x_pos_'):
        merged[column] = (merged[column] - field_start_x)/field_end_x
    elif column.startswith('y_pos_'):
        merged[column] = (merged[column] - field_start_y)/field_end_y
        
merged

,time,team,x_pos_1,x_pos_2,x_pos_3,x_pos_4,x_pos_5,x_pos_6,x_pos_7,y_pos_1,y_pos_2,y_pos_3,y_pos_4,y_pos_5,y_pos_6,y_pos_7,x_pos_ball,y_pos_ball,ballControl
0,5,0,0.286957,0.321739,0.455072,0.371014,0.111594,0.107246,NaN,0.732759,0.202586,0.443966,0.575431,0.463362,0.540948,NaN,0.504348,0.422414,True
1,5,1,0.626087,0.863768,0.665217,0.818841,0.698551,0.917391,0.621739,0.273707,0.571121,0.637931,0.487069,0.431034,0.519397,0.883621,0.504348,0.422414,False
2,6,0,0.321739,0.286957,0.111594,0.371014,0.107246,0.455072,NaN,0.202586,0.732759,0.463362,0.575431,0.540948,0.443966,NaN,0.504348,0.422414,True
3,6,1,0.621739,0.697101,0.917391,0.665217,0.863768,0.626087,0.818841,0.883621,0.431034,0.519397,0.640086,0.571121,0.275862,0.487069,0.504348,0.422414,False
4,7,0,0.455072,0.107246,0.321739,0.111594,0.286957,0.371014,NaN,0.443966,0.540948,0.202586,0.463362,0.732759,0.575431,NaN,0.504348,0.422414,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1665,1008,1,0.595652,0.850725,0.830435,0.871014,0.776812,0.597101,NaN,0.924569,0.413793,0.898707,0.760776,0.877155,0.730603,NaN,0.868116,0.672414,False
1666,1009,0,0.302899,0.113043,0.571014,0.372464,0.920290,NaN,NaN,0.545259,0.512931,0.452586,0.745690,0.551724,NaN,NaN,0.869565,0.670259,False
1667,1009,1,0.830435,0.595652,0.871014,0.850725,0.597101,NaN,NaN,0.898707,0.924569,0.760776,0.413793,0.730603,NaN,NaN,0.869565,0.670259,False
1668,1010,0,0.572464,0.372464,0.302899,0.113043,NaN,NaN,NaN,0.452586,0.745690,0.545259,0.508621,NaN,NaN,NaN,0.872464,0.665948,False


In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def sum_matrices_with_center(matrix_large, matrix_small, center_large):

    rows_large, cols_large = matrix_large.shape
    rows_small, cols_small = matrix_small.shape

    # Calculate the starting position of the smaller matrix within the larger matrix
    start_row = center_large[0] - rows_small // 2
    start_col = center_large[1] - cols_small // 2

    # Iterate through the smaller matrix and add elements to the larger matrix if indices are valid
    for i in range(rows_small):
        for j in range(cols_small):
            row_large = start_row + i
            col_large = start_col + j

            if 0 <= row_large < rows_large and 0 <= col_large < cols_large:
                matrix_large[row_large, col_large] += matrix_small[i, j]

    return matrix_large


def generateGaussian(img, cx, cy, sigma=3):
    size = 6*sigma + 3

    x = np.arange(0, size, 1, float)
    y = x[:, np.newaxis]
    x0, y0 = 3*sigma + 1, 3*sigma + 1
    g = np.exp(- ((x - x0) ** 2 + (y - y0) ** 2) / (2 * sigma ** 2))
    return sum_matrices_with_center(img, g, (cy, cx))



In [ ]:
from dataset import MyDataset
from torch.utils.data import DataLoader
from generator.utils_ import Distribution


ds = MyDataset(csv_file="merged.csv")
dl = DataLoader(ds, batch_size=32, shuffle=True)

matrix = next(iter(dl))[0, :, :].numpy()
Distribution.plot(matrix)
